## generate narrative from resource

### import modules

In [1]:
from fhir.resources import construct_fhir_element
from json import dumps, loads
from requests import get, post, put
from pathlib import Path
from IPython.display import display as Display, HTML, Markdown, Javascript
import ipywidgets as widgets
from pprint import pprint
from collections import namedtuple
from pandas import *
from datetime import datetime, date, timezone, timedelta
from jinja2 import Environment, FileSystemLoader, select_autoescape
from stringcase import snakecase, titlecase
from commonmark import commonmark
from htmlmin import minify

### Define variables

#### get CapStatement as json

In [2]:
fname = input('enter the CapStatment file name:')

enter the CapStatment file name: CapabilityStatement-backport-subscription-server.json


In [3]:
path = Path.cwd() / 'cs_source_file'/ fname
cs_py = construct_fhir_element('CapabilityStatement',path.read_text())
print(cs_py.id)

backport-subscription-server


### Create Profile url:Name mappings

In [4]:
# this is wrong if for most IGs should be supportedProfiles not Profiles
pname_map = {r.profile:titlecase(r.profile.split('/')[-1]) for r in cs_py.rest[0].resource if r.profile}
#add in supported Profiles too
spname_map = {sp:titlecase(sp.split('/')[-1]) for \
              r in cs_py.rest[0].resource if r.supportedProfile for sp in r.supportedProfile}
#flatten_matrix = [val for sublist in matrix for val in sublist] 

# Adding elements from dict2 to dict1
pname_map.update(spname_map)
pname_map

{'http://hl7.org/fhir/uv/subscriptions-backport/StructureDefinition/backport-subscription': 'Backport Subscription'}

### get pathmap from package file
- Get spec_internal from package a json file which includes canonical to local relative page links

In [6]:
ig_package = input("Get package path to download:")

Get package path to download: http://build.fhir.org/ig/HL7/fhir-subscription-backport-ig/package.tgz


In [7]:
r = get(ig_package, verify=False)
r.status_code
#write to file
path = Path.cwd() /  'cs_source_file' / ig_package.split('/')[-1]
path.write_bytes(r.content)

44564

In [8]:
import tarfile

def get_si(tar_file):
    print(path)
    with tarfile.open(tar_file, mode='r') as tf:
        pprint(tf.getnames())
        f = tf.extractfile('package/other/spec.internals')
        r = f.read()
        return(loads(r))


si = get_si(path)
       
path_map = si['paths']


#can = 'http://hl7.org/fhir/us/carin/StructureDefinition/carin-bb-explanationofbenefit-inpatient-facility'
#path_map[can]
path_map

/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/CapStatement/cs_source_file/package.tgz
['package/package.json',
 'package/ig-r4.json',
 'package/ImplementationGuide-hl7.fhir.uv.subscriptions-backport.json',
 'package/CapabilityStatement-backport-subscription-server.json',
 'package/example/Bundle-notification-handshake.json',
 'package/OperationDefinition-backport-subscription-status.json',
 'package/CodeSystem-backport-content-code-system.json',
 'package/OperationDefinition-backport-subscription-events.json',
 'package/example/Bundle-notification-id-only.json',
 'package/OperationDefinition-backport-subscription-get-ws-binding-token.json',
 'package/StructureDefinition-backport-channel-type.json',
 'package/example/Subscription-subscription-multi-resource.json',
 'package/StructureDefinition-backport-subscription-notification.json',
 'package/StructureDefinition-backport-heartbeat-period.json',
 'package/example/SubscriptionStatus-notification-status.json',
 'package/example/Bundl

{'http://hl7.org/fhir/uv/subscriptions-backport/ImplementationGuide/hl7.fhir.uv.subscriptions-backport|0.1.0': '0.1.0/ImplementationGuide-hl7.fhir.uv.subscriptions-backport.html',
 'http://hl7.org/fhir/uv/subscriptions-backport/ImplementationGuide/hl7.fhir.uv.subscriptions-backport': 'ImplementationGuide-hl7.fhir.uv.subscriptions-backport.html',
 'http://hl7.org/fhir/uv/subscriptions-backport/CapabilityStatement/CapabilitySubscriptionServer': 'CapabilityStatement-backport-subscription-server.html',
 'http://hl7.org/fhir/uv/subscriptions-backport/CapabilityStatement/CapabilitySubscriptionServer|0.1.0': '0.1.0/CapabilityStatement-backport-subscription-server.html',
 'http://hl7.org/fhir/uv/subscriptions-backport/CapabilityStatement/backport-subscription-server': 'CapabilityStatement-backport-subscription-server.html',
 'http://hl7.org/fhir/uv/subscriptions-backport/Bundle/notification-handshake': 'Bundle-notification-handshake.html',
 'http://hl7.org/fhir/uv/subscriptions-backport/Operat

### Get URL:Name/Title mapping from package file

In [9]:
from json import JSONDecodeError
def get_name_map(tar_file):
    print(path)
    name_map = {}
    with tarfile.open(tar_file, mode='r') as tf:
        for p_file in tf.getnames():
            #print(p_file)
            f = tf.extractfile(p_file)
            r = f.read()
            #print(type(r))
            try:
                r_dict = loads(r)
            except JSONDecodeError: # not a json file
                continue
            try:
                name_map[r_dict['url']] = r_dict['title']
                #print(r_dict['title'])
            except KeyError:
                try:
                    name_map[r_dict['url']] = r_dict['name']
                    #print(r_dict['name'])
                except KeyError: # not a conf resource
                    print(f'****{p_file}is not a conf resource***')
        return(name_map)


name_map = get_name_map(path)
name_map       

/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/CapStatement/cs_source_file/package.tgz
****package/example/Bundle-notification-handshake.jsonis not a conf resource***
****package/example/Bundle-notification-id-only.jsonis not a conf resource***
****package/example/Subscription-subscription-multi-resource.jsonis not a conf resource***
****package/example/SubscriptionStatus-notification-status.jsonis not a conf resource***
****package/example/Bundle-notification-empty.jsonis not a conf resource***
****package/example/Bundle-notification-error.jsonis not a conf resource***
****package/example/Subscription-subscription-admission.jsonis not a conf resource***
****package/example/Bundle-notification-full-resource.jsonis not a conf resource***
****package/example/Bundle-notification-multi-resource.jsonis not a conf resource***
****package/example/Bundle-notification-heartbeat.jsonis not a conf resource***
****package/openapi/backport-subscription-server.openapi.jsonis not a conf resource**

{'https://build.fhir.org/ig/HL7/fhir-subscription-backport-ig/': 'Subscriptions R5 Backport',
 'http://hl7.org/fhir/uv/subscriptions-backport/ImplementationGuide/hl7.fhir.uv.subscriptions-backport': 'Subscriptions R5 Backport',
 'http://hl7.org/fhir/uv/subscriptions-backport/CapabilityStatement/CapabilitySubscriptionServer': 'BackportSubscriptionCapabilityStatement',
 'http://hl7.org/fhir/uv/subscriptions-backport/OperationDefinition/OperationBackportSubscriptionStatus': 'R5SubscriptionStatus',
 'http://hl7.org/fhir/uv/subscriptions-backport/CodeSystem/backport-content-code-system': 'Backported R5 Subscription Content Code System',
 'http://hl7.org/fhir/uv/subscriptions-backport/OperationDefinition/OperationBackportSubscriptionEvents': 'R5SubscriptionEvents',
 'http://hl7.org/fhir/uv/subscriptions-backport/OperationDefinition/OperationBackportSubscriptionGetWsBindingToken': 'R5SubscriptionGetWsBindingToken',
 'http://hl7.org/fhir/uv/subscriptions-backport/StructureDefinition/backport-c

### Get SearchParameter parameter name: type mapping from package file

In [15]:
def get_sp_map(tar_file):
    print(path)
    sp_map = {}
    with tarfile.open(tar_file, mode='r') as tf:
        for p_file in tf.getnames():
            #print(p_file)
            if 'searchparameter' in p_file.lower():
                print(f'**********{p_file}')
                f = tf.extractfile(p_file)
                r = f.read()
                #print(type(r))
                try:
                    r_dict = loads(r)
                except JSONDecodeError: # not a json file
                    continue
                try:
                    sp_map[r_dict['code']] = r_dict['type']
                    print(r_dict['type'])
                except KeyError:
                    print(f'{p_file} has no type')
        return(sp_map)


sp_map = get_sp_map(path)
sp_map       

/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/CapStatement/cs_source_file/package.tgz


{}

### render using Jinja2 templates

create all maps:
- purl_map (canon:url)
- cs_map just canon:canon for now 
- ig_map just canon:canon for now 

In [16]:
in_path = ''
in_file = 'R4capabilitystatement-server.j2'

def markdown(text, *args, **kwargs):
    return commonmark(text, *args, **kwargs)

env = Environment(
    loader=FileSystemLoader(searchpath = in_path),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'],),
    trim_blocks = True,
    lstrip_blocks = True,
    )

env.filters['markdown'] = markdown

template = env.get_template(in_file)

purl_map={} # canon:url, 
for p,n in pname_map.items():
    try:
        purl_map[p] = pname_map[p]
    except KeyError:
        purl_map[p] = p
try:
    cs_list = cs_py.instantiates + cs_py.imports
except TypeError:
    cs_list =[]
cs_map = {k:k for k in cs_list} # just canon:canon for now 
#-TODO see how can look up fhir registry and populate
ig_map = {k:k for k in cs_py.implementationGuide}

rendered = template.render(cs=cs_py, path_map=path_map, pname_map=name_map, purl_map=purl_map, sp_map=sp_map, 
                          csname_map=cs_map, csurl_map=cs_map, igname_map=ig_map, igurl_map=ig_map)

### Minify the xhtml

In [17]:
def x_minify(xhtml):
    h_min=minify(xhtml, remove_optional_attribute_quotes=False, remove_comments=True)
    x_min = h_min.replace('<br>','<br />')
    x_min = x_min.replace('<hr>','<hr />')
    return x_min

mini = x_minify(rendered)
#print(type(mini))
#print(mini)
#display(HTML(rendered))
display(HTML(mini))

'''
#======== write to temp file to debug =======
path = Path.cwd() / 'debug' / 'narrative_pre.xhtml'
path.write_text(rendered, encoding="utf-8")
path = Path.cwd() / 'debug' / 'narrative_mini_pre.xhtml'
path.write_text(mini, encoding="utf-8")
#===================================================
'''
narr = construct_fhir_element('Narrative',dict(
    status = 'generated',
    div = mini,
 ))
cs_py.text = narr

### Validation

In [18]:

def validate(r):
    #fhir_test_server = 'http://fhirtest.uhn.ca/baseDstu3'
    #fhir_test_server = 'http://test.fhir.org/r3'
    #fhir_test_server = 'http://test.fhir.org/r4'
    #fhir_test_server ='http://wildfhir4.aegis.net/fhir4-0-0'
    #fhir_test_server ='http://wildfhir4.aegis.net/fhir4-0-0'
    fhir_test_server ='http://hapi.fhir.org/baseR4'
  

    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }
    
    r = post(f'{fhir_test_server}/CapabilityStatement/$validate', headers = headers, data = cs_py.json())   # return r.status_code
    display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))

validate(cs_py)

WARNING,"[CapabilityStatement.jurisdiction[0], Line 1, Col 4497]","None of the codings provided are in the value set 'Jurisdiction' (http://hl7.org/fhir/ValueSet/jurisdiction), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://unstats.un.org/unsd/methods/m49/m49.htm#001)"
ERROR,"[CapabilityStatement.fhirVersion, Line 1, Col 4627]","The value provided ('4.1.0') is not in the value set 'FHIRVersion' (http://hl7.org/fhir/ValueSet/FHIR-version|4.0.1), and a code is required from this value set) (error message = Unknown code '4.1.0' for in-memory expansion of ValueSet 'http://hl7.org/fhir/ValueSet/FHIR-version')"
ERROR,"[CapabilityStatement.rest[0].resource[0].type, Line 1, Col 4959]","The value provided ('SubscriptionTopic') is not in the value set 'ResourceType' (http://hl7.org/fhir/ValueSet/resource-types|4.0.1), and a code is required from this value set) (error message = Unknown code 'SubscriptionTopic' for in-memory expansion of ValueSet 'http://hl7.org/fhir/ValueSet/resource-types')"


### Save to local dir

In [19]:
# save to file
ig_source_path = ''
print('...........saving to file............')
#save in ig_source folder
path = Path.cwd() / ig_source_path / 'resources' / f'capabilitystatement-{cs_py.id.lower()}.json'
path.write_text(cs_py.json(indent=2))

...........saving to file............


12973